In [214]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/machine-learning-hackathon/sample_submission.csv/sample_submission.csv
/kaggle/input/machine-learning-hackathon/train.csv/train.csv
/kaggle/input/machine-learning-hackathon/test.csv/test.csv


In [215]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix
from xgboost import XGBClassifier
import seaborn as sns
import pickle
%matplotlib inline

In [216]:
df = pd.read_csv("../input/machine-learning-hackathon/train.csv/train.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [217]:
df.head()

,id,impr_time,campaign_type,page_type,campaign_id,pid,cid,mcid,ch,at,slot,cpc,uuid,is_click,click_time
0,1,2017-09-10 00:00:01,CATEGORY,PDP,21990,125382836,1557,282,A,R,2,3.0,230a1a0cc6401f94,0,NaN
1,2,2017-09-10 00:00:01,CATEGORY,PDP,21997,121438156,1557,282,A,R,1,3.0,230a1a0cc6401f94,0,NaN
2,3,2017-09-10 00:00:01,CATEGORY,PDP,21930,126450004,1557,282,A,R,3,2.0,230a1a0cc6401f94,0,NaN
3,5,2017-09-10 00:00:01,CATEGORY,PDP,21932,126451041,1557,282,A,R,5,2.0,230a1a0cc6401f94,0,NaN
4,6,2017-09-10 00:00:01,CATEGORY,PDP,21931,81731596,1557,282,A,R,4,2.0,230a1a0cc6401f94,0,NaN


In [218]:
df.tail()

,id,impr_time,campaign_type,page_type,campaign_id,pid,cid,mcid,ch,at,slot,cpc,uuid,is_click,click_time
2143880,2679222,2017-09-10 23:59:55,CATEGORY,PDP,21972,105976049,247,89,M,R,1,1.5,501628FEB4874AE086BBBACD7F0FCA9A,0,NaN
2143881,2679223,2017-09-10 23:59:55,CATEGORY,PDP,21973,118610664,247,89,M,R,2,1.5,501628FEB4874AE086BBBACD7F0FCA9A,0,NaN
2143882,2679224,2017-09-10 23:59:57,CATEGORY,PDP,21828,110014318,678,32644,M,R,1,3.0,0921D2282A994A0086CBDFBE5466E1B0,0,NaN
2143883,2679225,2017-09-10 23:59:57,CATEGORY,PDP,21824,110014246,678,32644,M,R,2,3.0,0921D2282A994A0086CBDFBE5466E1B0,0,NaN
2143884,2679226,2017-09-10 23:59:59,CATEGORY,PDP,19064,101922679,3690,87,M,R,1,2.5,undefined,0,NaN


In [219]:
df["is_click"].unique()

array([0, 1])

In [220]:
df["is_click"].value_counts()

0    2122587
1      21298
Name: is_click, dtype: int64

Imbalanced

*Is Click contains 0 and 1 . ) means not clicked and 1 means clicked*

In [221]:
df["campaign_type"].unique()

array(['CATEGORY', 'SPB'], dtype=object)

In [222]:
df["page_type"].unique()

array(['PDP', 'CATEGORY', 'undefined'], dtype=object)

In [223]:
df["ch"].unique()

array(['A', 'D', 'M'], dtype=object)

In [224]:
df["at"].unique()

array(['R', 'N', 'B'], dtype=object)

Categorical Features for 1 hot encoding (less # unique) - campaign Type, Page Type, Ch Channel, At Channel <br>
Categorical Features for 1 hot encoding (more # unique)

In [225]:
df.shape

(2143885, 15)

# Dropping Null Values

In [226]:
df.isnull().sum()

id                     0
impr_time              0
campaign_type          0
page_type              0
campaign_id            0
pid                    0
cid                    0
mcid                   0
ch                     0
at                     0
slot                   0
cpc                    0
uuid                  11
is_click               0
click_time       2122587
dtype: int64

## as there are only 11 rows with null values, we can remove the rows directly

In [227]:
df=df.dropna(axis=0, subset=["uuid"])

In [228]:
df.isnull().sum()

id                     0
impr_time              0
campaign_type          0
page_type              0
campaign_id            0
pid                    0
cid                    0
mcid                   0
ch                     0
at                     0
slot                   0
cpc                    0
uuid                   0
is_click               0
click_time       2122576
dtype: int64

In [229]:
df["is_click"].value_counts()

0    2122576
1      21298
Name: is_click, dtype: int64

In [230]:
df= pd.get_dummies(df, columns=["campaign_type","page_type","ch","at"])

In [231]:
df["is_click"].value_counts()

0    2122576
1      21298
Name: is_click, dtype: int64

In [232]:
df.head()

,id,impr_time,campaign_id,pid,cid,mcid,slot,cpc,uuid,is_click,...,campaign_type_SPB,page_type_CATEGORY,page_type_PDP,page_type_undefined,ch_A,ch_D,ch_M,at_B,at_N,at_R
0,1,2017-09-10 00:00:01,21990,125382836,1557,282,2,3.0,230a1a0cc6401f94,0,...,0,0,1,0,1,0,0,0,0,1
1,2,2017-09-10 00:00:01,21997,121438156,1557,282,1,3.0,230a1a0cc6401f94,0,...,0,0,1,0,1,0,0,0,0,1
2,3,2017-09-10 00:00:01,21930,126450004,1557,282,3,2.0,230a1a0cc6401f94,0,...,0,0,1,0,1,0,0,0,0,1
3,5,2017-09-10 00:00:01,21932,126451041,1557,282,5,2.0,230a1a0cc6401f94,0,...,0,0,1,0,1,0,0,0,0,1
4,6,2017-09-10 00:00:01,21931,81731596,1557,282,4,2.0,230a1a0cc6401f94,0,...,0,0,1,0,1,0,0,0,0,1


# Applying same preprocessing in test data to make the data similar

In [233]:
dftest = pd.read_csv("../input/machine-learning-hackathon/test.csv/test.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [234]:
dftest.head()

,id,impr_time,campaign_type,page_type,campaign_id,pid,cid,mcid,ch,at,slot,cpc,uuid
0,4,2017-09-10 00:00:01,CATEGORY,PDP,21933,126450627,1557,282,A,R,6,2.0,230a1a0cc6401f94
1,10,2017-09-10 00:00:01,CATEGORY,PDP,21937,81719978,1557,282,A,R,10,2.0,230a1a0cc6401f94
2,12,2017-09-10 00:00:01,CATEGORY,PDP,21717,123995949,62386,480,A,R,2,5.5,7eb97ba3ac3d5142
3,15,2017-09-10 00:00:01,CATEGORY,PDP,19272,97946679,62386,480,A,R,7,4.0,7eb97ba3ac3d5142
4,19,2017-09-10 00:00:02,CATEGORY,PDP,18525,102213695,62386,480,A,R,10,3.0,7eb97ba3ac3d5142


In [235]:
#print(dftest["is_click"].unique())
print(dftest["campaign_type"].unique())
print(dftest["page_type"].unique())
print(dftest["ch"].unique())
print(dftest["at"].unique())

['CATEGORY' 'SPB']
['PDP' 'CATEGORY' 'undefined']
['A' 'M' 'D']
['R' 'N' 'B']


no category is missing in test data so we may proceed

In [236]:
dftest.isnull().sum()

id               0
impr_time        0
campaign_type    0
page_type        0
campaign_id      0
pid              0
cid              0
mcid             0
ch               0
at               0
slot             0
cpc              0
uuid             0
dtype: int64

no null values

In [237]:
dftest= pd.get_dummies(dftest, columns=["campaign_type","page_type","ch","at"])

we do not need to match the column numbering as in one hot its only 0 or 1 representation for every column


In [238]:
dftest.head()

,id,impr_time,campaign_id,pid,cid,mcid,slot,cpc,uuid,campaign_type_CATEGORY,campaign_type_SPB,page_type_CATEGORY,page_type_PDP,page_type_undefined,ch_A,ch_D,ch_M,at_B,at_N,at_R
0,4,2017-09-10 00:00:01,21933,126450627,1557,282,6,2.0,230a1a0cc6401f94,1,0,0,1,0,1,0,0,0,0,1
1,10,2017-09-10 00:00:01,21937,81719978,1557,282,10,2.0,230a1a0cc6401f94,1,0,0,1,0,1,0,0,0,0,1
2,12,2017-09-10 00:00:01,21717,123995949,62386,480,2,5.5,7eb97ba3ac3d5142,1,0,0,1,0,1,0,0,0,0,1
3,15,2017-09-10 00:00:01,19272,97946679,62386,480,7,4.0,7eb97ba3ac3d5142,1,0,0,1,0,1,0,0,0,0,1
4,19,2017-09-10 00:00:02,18525,102213695,62386,480,10,3.0,7eb97ba3ac3d5142,1,0,0,1,0,1,0,0,0,0,1


In [239]:
df.head()

,id,impr_time,campaign_id,pid,cid,mcid,slot,cpc,uuid,is_click,...,campaign_type_SPB,page_type_CATEGORY,page_type_PDP,page_type_undefined,ch_A,ch_D,ch_M,at_B,at_N,at_R
0,1,2017-09-10 00:00:01,21990,125382836,1557,282,2,3.0,230a1a0cc6401f94,0,...,0,0,1,0,1,0,0,0,0,1
1,2,2017-09-10 00:00:01,21997,121438156,1557,282,1,3.0,230a1a0cc6401f94,0,...,0,0,1,0,1,0,0,0,0,1
2,3,2017-09-10 00:00:01,21930,126450004,1557,282,3,2.0,230a1a0cc6401f94,0,...,0,0,1,0,1,0,0,0,0,1
3,5,2017-09-10 00:00:01,21932,126451041,1557,282,5,2.0,230a1a0cc6401f94,0,...,0,0,1,0,1,0,0,0,0,1
4,6,2017-09-10 00:00:01,21931,81731596,1557,282,4,2.0,230a1a0cc6401f94,0,...,0,0,1,0,1,0,0,0,0,1


In [240]:
df.shape

(2143874, 22)

In [241]:
dftest.shape

(535342, 20)

In [242]:
list(df)

['id',
 'impr_time',
 'campaign_id',
 'pid',
 'cid',
 'mcid',
 'slot',
 'cpc',
 'uuid',
 'is_click',
 'click_time',
 'campaign_type_CATEGORY',
 'campaign_type_SPB',
 'page_type_CATEGORY',
 'page_type_PDP',
 'page_type_undefined',
 'ch_A',
 'ch_D',
 'ch_M',
 'at_B',
 'at_N',
 'at_R']

In [243]:
list(dftest)

['id',
 'impr_time',
 'campaign_id',
 'pid',
 'cid',
 'mcid',
 'slot',
 'cpc',
 'uuid',
 'campaign_type_CATEGORY',
 'campaign_type_SPB',
 'page_type_CATEGORY',
 'page_type_PDP',
 'page_type_undefined',
 'ch_A',
 'ch_D',
 'ch_M',
 'at_B',
 'at_N',
 'at_R']

In [244]:
df=df.drop(["click_time"], axis=1)

In [245]:
df.shape

(2143874, 21)

# Train Test Valid Datas X and Y

In [246]:
df["is_click"].value_counts()

0    2122576
1      21298
Name: is_click, dtype: int64

In [247]:
df["is_click"].unique()

array([0, 1])

In [248]:
df.dtypes

id                          int64
impr_time                  object
campaign_id                 int64
pid                        object
cid                        object
mcid                       object
slot                        int64
cpc                       float64
uuid                       object
is_click                    int64
campaign_type_CATEGORY      uint8
campaign_type_SPB           uint8
page_type_CATEGORY          uint8
page_type_PDP               uint8
page_type_undefined         uint8
ch_A                        uint8
ch_D                        uint8
ch_M                        uint8
at_B                        uint8
at_N                        uint8
at_R                        uint8
dtype: object

In [249]:
df["pid"]=pd.to_numeric(df["pid"], errors="coerce")

In [250]:
df["cid"]=pd.to_numeric(df["cid"], errors="coerce")
df["mcid"]=pd.to_numeric(df["mcid"], errors="coerce")

In [251]:
dftest["pid"]=pd.to_numeric(dftest["pid"], errors="coerce")
dftest["cid"]=pd.to_numeric(dftest["cid"], errors="coerce")
dftest["mcid"]=pd.to_numeric(dftest["mcid"], errors="coerce")

In [252]:
df.dtypes

id                          int64
impr_time                  object
campaign_id                 int64
pid                       float64
cid                       float64
mcid                      float64
slot                        int64
cpc                       float64
uuid                       object
is_click                    int64
campaign_type_CATEGORY      uint8
campaign_type_SPB           uint8
page_type_CATEGORY          uint8
page_type_PDP               uint8
page_type_undefined         uint8
ch_A                        uint8
ch_D                        uint8
ch_M                        uint8
at_B                        uint8
at_N                        uint8
at_R                        uint8
dtype: object

In [253]:
dftest.dtypes

id                          int64
impr_time                  object
campaign_id                 int64
pid                       float64
cid                       float64
mcid                      float64
slot                        int64
cpc                       float64
uuid                       object
campaign_type_CATEGORY      uint8
campaign_type_SPB           uint8
page_type_CATEGORY          uint8
page_type_PDP               uint8
page_type_undefined         uint8
ch_A                        uint8
ch_D                        uint8
ch_M                        uint8
at_B                        uint8
at_N                        uint8
at_R                        uint8
dtype: object

In [254]:
df = df.drop(["impr_time"], axis=1)
dftest = dftest.drop(["impr_time"], axis=1)

In [255]:
df.head()

,id,campaign_id,pid,cid,mcid,slot,cpc,uuid,is_click,campaign_type_CATEGORY,campaign_type_SPB,page_type_CATEGORY,page_type_PDP,page_type_undefined,ch_A,ch_D,ch_M,at_B,at_N,at_R
0,1,21990,125382836.0,1557.0,282.0,2,3.0,230a1a0cc6401f94,0,1,0,0,1,0,1,0,0,0,0,1
1,2,21997,121438156.0,1557.0,282.0,1,3.0,230a1a0cc6401f94,0,1,0,0,1,0,1,0,0,0,0,1
2,3,21930,126450004.0,1557.0,282.0,3,2.0,230a1a0cc6401f94,0,1,0,0,1,0,1,0,0,0,0,1
3,5,21932,126451041.0,1557.0,282.0,5,2.0,230a1a0cc6401f94,0,1,0,0,1,0,1,0,0,0,0,1
4,6,21931,81731596.0,1557.0,282.0,4,2.0,230a1a0cc6401f94,0,1,0,0,1,0,1,0,0,0,0,1


In [256]:
df = df.drop(["uuid"], axis=1)
dftest = dftest.drop(["uuid"], axis=1)

In [257]:
df.shape

(2143874, 19)

In [258]:
dftest.shape

(535342, 18)

In [259]:
Y = df["is_click"]
X = df.drop(["is_click"], axis=1)

In [260]:
Y

0          0
1          0
2          0
3          0
4          0
          ..
2143880    0
2143881    0
2143882    0
2143883    0
2143884    0
Name: is_click, Length: 2143874, dtype: int64

In [261]:
X

,id,campaign_id,pid,cid,mcid,slot,cpc,campaign_type_CATEGORY,campaign_type_SPB,page_type_CATEGORY,page_type_PDP,page_type_undefined,ch_A,ch_D,ch_M,at_B,at_N,at_R
0,1,21990,125382836.0,1557.0,282.0,2,3.0,1,0,0,1,0,1,0,0,0,0,1
1,2,21997,121438156.0,1557.0,282.0,1,3.0,1,0,0,1,0,1,0,0,0,0,1
2,3,21930,126450004.0,1557.0,282.0,3,2.0,1,0,0,1,0,1,0,0,0,0,1
3,5,21932,126451041.0,1557.0,282.0,5,2.0,1,0,0,1,0,1,0,0,0,0,1
4,6,21931,81731596.0,1557.0,282.0,4,2.0,1,0,0,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2143880,2679222,21972,105976049.0,247.0,89.0,1,1.5,1,0,0,1,0,0,0,1,0,0,1
2143881,2679223,21973,118610664.0,247.0,89.0,2,1.5,1,0,0,1,0,0,0,1,0,0,1
2143882,2679224,21828,110014318.0,678.0,32644.0,1,3.0,1,0,0,1,0,0,0,1,0,0,1
2143883,2679225,21824,110014246.0,678.0,32644.0,2,3.0,1,0,0,1,0,0,0,1,0,0,1


In [262]:
X.shape

(2143874, 18)

In [263]:
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, random_state=0)

In [264]:
X_train.shape

(1607905, 18)

In [265]:
X_valid.shape

(535969, 18)

In [266]:
X_train.dtypes

id                          int64
campaign_id                 int64
pid                       float64
cid                       float64
mcid                      float64
slot                        int64
cpc                       float64
campaign_type_CATEGORY      uint8
campaign_type_SPB           uint8
page_type_CATEGORY          uint8
page_type_PDP               uint8
page_type_undefined         uint8
ch_A                        uint8
ch_D                        uint8
ch_M                        uint8
at_B                        uint8
at_N                        uint8
at_R                        uint8
dtype: object

In [267]:
X_valid.dtypes

id                          int64
campaign_id                 int64
pid                       float64
cid                       float64
mcid                      float64
slot                        int64
cpc                       float64
campaign_type_CATEGORY      uint8
campaign_type_SPB           uint8
page_type_CATEGORY          uint8
page_type_PDP               uint8
page_type_undefined         uint8
ch_A                        uint8
ch_D                        uint8
ch_M                        uint8
at_B                        uint8
at_N                        uint8
at_R                        uint8
dtype: object

In [268]:
y_train.dtypes

dtype('int64')

In [269]:
y_valid.dtypes

dtype('int64')

# Model - XGBOOST

## Why XG Boost - Because data is imbalanced

In [270]:
Y.describe()

count    2.143874e+06
mean     9.934352e-03
std      9.917492e-02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: is_click, dtype: float64

In [271]:
X_train.shape

(1607905, 18)

In [272]:
y_train.shape

(1607905,)

In [273]:
X_valid.shape

(535969, 18)

In [274]:
y_valid.shape

(535969,)

In [275]:
y_train=pd.DataFrame(y_train)

In [276]:
y_valid=pd.DataFrame(y_valid)

In [277]:
y_train.head()

,is_click
638689,0
1890139,0
1570951,0
94211,0
250043,0


In [278]:
y_train.shape

(1607905, 1)

In [279]:
from xgboost import XGBRegressor

my_model_1 = XGBRegressor(random_state=0)
my_model_1.fit(X_train, y_train) 

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [280]:
predictions_1 = my_model_1.predict(X_valid)

In [281]:
predictions_1

array([0.0371399 , 0.01765886, 0.00377678, ..., 0.01657058, 0.00409346,
       0.00362744], dtype=float32)

In [282]:
from sklearn.metrics import mean_absolute_error
# Calculate MAE
mae_1 = mean_absolute_error(predictions_1, y_valid) # Your code here

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_1)

Mean Absolute Error: 0.0194069184993578


In [283]:
dftest.shape

(535342, 18)

In [284]:
dftest.dtypes

id                          int64
campaign_id                 int64
pid                       float64
cid                       float64
mcid                      float64
slot                        int64
cpc                       float64
campaign_type_CATEGORY      uint8
campaign_type_SPB           uint8
page_type_CATEGORY          uint8
page_type_PDP               uint8
page_type_undefined         uint8
ch_A                        uint8
ch_D                        uint8
ch_M                        uint8
at_B                        uint8
at_N                        uint8
at_R                        uint8
dtype: object

In [285]:
predictions_2 = my_model_1.predict(dftest)
output = pd.DataFrame({'id': dftest.id,
                       'is_click': predictions_2})
output.to_csv('submission.csv', index=False)